# This file is generally to work out the absorption coefficient $I(\vec{k},\vec{G})$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erf
import astropy.units as unit
import astropy.constants as const
import itertools

In [2]:
#Define known physical constants
a_nai = 6.642 #Lattice constant, Angstroms = 1e-10m, from Dent paper. 
hbar_kevs = (const.hbar.to(unit.keV*unit.s)).value #hbar, keV*s, from astropy
c_aas = (const.c.to(unit.AA/unit.s)).value #speed of light, angstrom/s (cancels with a), from astropy
me_hca = (const.m_e/((const.hbar/(const.c*unit.AA)).si)).value #Electron mass, in units of... hbar/(c*AA) to cancel with |G| (LMAOOOOO)

In [3]:
# Define unknown constants/crystal info
g_agg = 1e-10 #g_agg (axion-photon-photon) coupling constant; in GeV^(-1) (for photon coalescence)
g_ag = 1e-10 #Axion-photon coupling (Primakoff process), in GeV^(-1).
#Upper bound on g_ag of 2.7e-10 from the Sun, see Di Luzio's review
lmda = (g_agg/(1e-8))**4
phi_0 = 5.95e14 #in cm^(-2) s^(-1)
E_0    = 1.103 #in keV

#Crystal Geometry
diam = 110*unit.mm
hgt = 150*unit.mm 
vol = ((np.pi*(diam/2)**2*hgt).to(unit.AA**3)).value #Crystal volume, in AA^3

vol_cell = 67.71 #In AA^3, from Dent paper

In [4]:
#Define useful functions

def make_hkl(max_h=5,max_k=5,max_l=5, s=False):
    output_list = []
    for h in range(max_h+1):
        for k in range(max_k+1):
            if k%2 == h%2:
                for l in range(max_l+1):
                    if l%2 == h%2:
                        if s:
                            if l%2 == 0:
                                output_list.append((h,k,l))
                        else:
                            output_list.append((h,k,l))
    return np.array(output_list[1:])

def mod2(vec3):
    '''Returns the dimensionless magnitude of an array of vectors in the (h,k,l) basis. Factor of 2pi/a has to be multiplied manually'''
    return 3*(vec3[:,0]**2 + vec3[:,1]**2 + vec3[:,2]**2) - 2*(vec3[:,0]*vec3[:,1] + vec3[:,1]*vec3[:,2] + vec3[:,2]*vec3[:,0])

def gdotk(hkl_arr):
    '''Returns gHAT dot kHAT'''
    return (-hkl_arr[:,0]+hkl_arr[:,1]+hkl_arr[:,2])/np.sqrt(mod2(hkl_arr)) #IJBOL -- you can't just say ijbol

def w_func(Ea, dVector = False, Delta=1.5, E1=1, E2=6):
    '''Returns array of W(Ea, Delta, E1, E2) where everything is in keV. Infinite Ea is dealt with'''
    w_list = []
    if dVector:
        entangle = np.dstack((Ea, Delta))
        for index in range(entangle.shape[1]):
            E = entangle[0,index,0]
            D = entangle[0,index,1]
            if np.isinf(E):
                w_list.append(0)
            else:
                w = (1/2)*(erf((E-E1)/(np.sqrt(2)*D)) - erf((E-E2)/(np.sqrt(2)*D)))
                w_list.append(w)
    else:
        for E in Ea:
            if np.isinf(E):
                w_list.append(0)
            else:
                w = (1/2)*(erf((E-E1)/(np.sqrt(2)*Delta)) - erf((E-E2)/(np.sqrt(2)*Delta)))
                w_list.append(w)
    return np.array(w_list)

def make_dpde(Energy_array):
    '''Creates dPhi/dE for the parametrised blackbody-like form appearing in older papers. Assumes no axion mass.
    Units of cm^(-2) s^(-1) keV^(-1)'''
    dpdt_list = []
    for E in Energy_array:
        if np.isinf(E):
            dpdt_list.append(0)
        else:
            dpdt_list.append(np.sqrt(lmda)*(phi_0)/(E_0) * (E/E_0)**3/(np.exp(E/E_0)-1))
    return np.array(dpdt_list)

def hkl_to_cart(hkl_array):
    '''Converts from hkl basis to cartesian basis, ignoring factor of 2pi/a'''
    x_axis = - hkl_array[:,0] + hkl_array[:,1] + hkl_array[:,2] 
    y_axis =   hkl_array[:,0] - hkl_array[:,1] + hkl_array[:,2] 
    z_axis =   hkl_array[:,0] + hkl_array[:,1] - hkl_array[:,2] 
    return np.stack((x_axis, y_axis, z_axis), axis=1)

In [5]:
hkl = make_hkl(s=True)
hkl_c = hkl_to_cart(hkl)

In [6]:
g_cart = hkl_c*2*np.pi/a_nai #in AA^(-1)

In [7]:
def one_cube(a):
    return [(0,0,0),(a,0,0),(0,0,a),(a,0,a),
            (0,a,0),(a,a,0),(0,a,a),(a,a,a),
            (0,a/2,a/2), (a/2,0,a/2), (a/2,a/2,0),
            (a,a/2,a/2), (a/2,a,a/2), (a/2,a/2,a)]

In [74]:
def make_lattice_points(cell_vec = (10,10,10), a = a_nai):
    #Number of points is almost exactly O(n^3)
    nx = cell_vec[0]
    ny = cell_vec[1]
    nz = cell_vec[2]    

    all_points = np.array(one_cube(a)) #Initial cube
    init_list = W
    
    for start_vec in init_list:
        extend_table = np.repeat(np.array([start_vec]), 14, axis=0)
        points = np.array(one_cube(a)) + extend_table
        all_points = np.vstack((all_points, points))
    return np.unique(all_points, axis=0)

In [14]:
idk = 10*make_lattice_points(cell_vec=(7,
                                       7,
                                       7))

In [15]:
len(idk)

4802

In [16]:
idk

array([[  0.  ,   0.  ,   0.  ],
       [  0.  ,   0.  ,  10.  ],
       [  0.  ,   0.  ,  20.  ],
       ...,
       [126.42, 126.42, 106.42],
       [126.42, 126.42, 116.42],
       [126.42, 126.42, 126.42]])

In [17]:
erm = []

for vec1 in idk:
    for vec2 in idk:
        erm.append(vec1-vec2)

erm = np.array(erm)
#47.1s when n=7^3

In [18]:
erm

array([[  0.,   0.,   0.],
       [  0.,   0., -10.],
       [  0.,   0., -20.],
       ...,
       [  0.,   0.,  20.],
       [  0.,   0.,  10.],
       [  0.,   0.,   0.]])

In [73]:
erm.shape

(23059204, 3)

In [68]:
mask = np.abs(erm[:,0]) > 0
sorted = np.sort(np.exp(-10*erm[:,0]))
sorted

C:\Users\taase\AppData\Local\Temp\ipykernel_11324\3327782044.py:2: RuntimeWarning: overflow encountered in exp
  sorted = np.sort(np.exp(-10*erm[:,0]))


array([ 0.,  0.,  0., ..., inf, inf, inf])

In [69]:
np.sum(sorted)

inf

In [70]:
len(sorted)

23059204

In [71]:
sorted[sorted<1]

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
       1.14590109e-14, 1.14590109e-14, 1.14590109e-14])

In [72]:
np.sort(np.abs(erm[:,0]))

array([  0.  ,   0.  ,   0.  , ..., 126.42, 126.42, 126.42])